# 정보공개서 데이터 수집
- 출처 : 공정거래위원회 가맹사업거래 홈페이지 (https://franchise.ftc.go.kr/main/index.do)
- '가맹희망플러스' 탭에서 업종 '외식' 데이터 전부 수집 (2016년-2020년 등록 데이터)
- '정보공개서' 탭에서 2020년 데이터 수집 (2021년 등록)

# 1. 2016년 - 2020년 등록 데이터

In [21]:
def get_data(등록년도, 비교항목1, 비교항목2):
    """
    등록년도별 비교항목 수집 함수
    
    등록년도 : 2015-2020
    비교항목1 : 해당 페이지 첫번째 드롭다운(업종별, 가맹본부별, 브랜드별)
    비교항목2 : 해당 페이지 두번째 드롭다운

    """
    
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import Select
    from bs4 import BeautifulSoup
    import time
    import pandas as pd

    chromedriver = '/usr/local/bin/chromedriver/chromedriver'
    driver = webdriver.Chrome(chromedriver)

    #크롤링 사이트 호출
    url = 'https://franchise.ftc.go.kr/mnu/00014/program/firHope/view.do'
    driver.get(url)
    
    # 드롭다운 선택
    button0= Select(driver.find_element_by_css_selector('#selYear'))
    button0.select_by_visible_text(등록년도)
    button1= Select(driver.find_element_by_css_selector('#searchCondition'))
    button1.select_by_visible_text(비교항목1)
    button2 = Select(driver.find_element_by_css_selector('#selListType'))
    button2.select_by_visible_text(비교항목2)
    button3 = Select(driver.find_element_by_css_selector('#selUpjong'))
    button3.select_by_visible_text('외식')
    time.sleep(1)
    
    # 빈 데이터프레임 만들기
    new_df = pd.DataFrame()
    
    # 업종별 데이터프레임 만들기
    category = ['A1','B1','C1','D1','E1','F1','G1','H1','I1','J1','K1','L1','M1','N1','O1']
    for i in category :
        button4 = Select(driver.find_element_by_css_selector('#selIndus'))
        button4.select_by_value(i)
        
        # 검색 버튼
        search_button = driver.find_element_by_css_selector('#frm > div:nth-child(6) > input[type=image]:nth-child(3)')

        # 키보드/마우스 동작 자동화 > 검색 버튼 클릭
        webdriver.ActionChains(driver).click(search_button).perform()
        time.sleep(2)
        
        # table 형태 html 그대로 저장
        result_html = driver.page_source
        result_soup = BeautifulSoup(result_html, 'html.parser')
        tags = result_soup.find(attrs={"class":"box_plus kft_box"})
        
        ##!pip install html_table_parser
        from html_table_parser import parser_functions as parser
        html_table = parser.make2d(tags)

        df=pd.DataFrame(html_table[1:], columns=html_table[0])
        df['업종'] = i ## 업종 추가

        new_df = new_df.append(df) ## 업종별 데이터 합치기
        
    driver.quit()    
    return new_df

In [25]:
def get_df(비교항목1, 비교항목2):
    """
    필요한 연도별(2016-2020) 비교항목 데이터프레임 만드는 함수
    비교항목1 : 해당 페이지 첫번째 드롭다운(업종별, 가맹본부별, 브랜드별)
    비교항목2 : 해당 페이지 두번째 드롭다운

    """   
    import pandas as pd
    new_df = pd.DataFrame()
    years = ['2020', '2019', '2018', '2017', '2016']
    for year in years:
        df = get_data(year, 비교항목1, 비교항목2)
        df['기준년도'] = year
        new_df = pd.concat([new_df, df])
    return new_df

In [40]:
import pandas as pd

# 데이터 수집 시작
## 가맹본부별 > 성장성, 안정성, 수익성
growth = get_df('가맹본부별', '성장성')
stability = get_df('가맹본부별', '안정성')
profit = get_df('가맹본부별', '수익성')

## 브랜드별 > 브랜드 개요, 가맹정 현황 정보, 가맹점 창업비용
overview = get_df('브랜드별', '브랜드 개요')
info = get_df('브랜드별', '가맹점 현황 정보')
cost = get_df('브랜드별', '가맹점 창업비용')

In [41]:
# 확인
growth.shape, stability.shape, profit.shape, overview.shape, info.shape, cost.shape

((11109, 9), (11109, 9), (11109, 10), (20585, 8), (20426, 11), (20915, 12))

In [44]:
# csv 파일로 변환
growth.to_csv('growth.csv')
stability.to_csv('stability.csv')
profit.to_csv('profit.csv')
overview.to_csv('overview.csv')
info.to_csv('info.csv')
cost.to_csv('cost.csv')

# 2. 2021년 등록 데이터

In [241]:
# 정보공개서 개별 주소 크롤링

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

chromedriver = '/usr/local/bin/chromedriver/chromedriver'
driver = webdriver.Chrome(chromedriver)

ls = []
for num in range(1,24):
    #크롤링 사이트 호출
    url = 'https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do?searchCondition=&searchKeyword=&column=brd&selUpjong=21&selIndus=&pageUnit=300&pageIndex='+str(num)
    driver.get(url)
    time.sleep(2)

    elem = driver.find_elements_by_class_name('authCtrl')
    for i in elem:
        ls.append(i.get_attribute('onclick'))

ls = list(set(ls)) ## 중복제거

In [63]:
# 정보공개서 크롤링

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd

chromedriver = '/usr/local/bin/chromedriver/chromedriver'
driver = webdriver.Chrome(chromedriver)

total_df = pd.DataFrame()

#크롤링 사이트 호출
for site in ls:
    url = 'https://franchise.ftc.go.kr'+ site.split("'")[1]
    driver.get(url)

    # table 형태 html 그대로 저장
    result_html = driver.page_source
    result_soup = BeautifulSoup(result_html, 'html.parser')
    tags = result_soup.find_all(attrs={"class":"table"})

    from html_table_parser import parser_functions as parser
    ## 가맹본부 일반현황
    html_table = parser.make2d(tags[0])
    df=pd.DataFrame(html_table[1:], columns=html_table[0]).iloc[:1,:]
    ## 가맹본부 재무상황
    html_table = parser.make2d(tags[2])
    tmp=pd.DataFrame(html_table[1:], columns=html_table[0])
    df = pd.concat([df,tmp.iloc[:1,:]],axis=1)
    ## 가맹본부 임직원수
    html_table = parser.make2d(tags[3])
    try:
        df['가맹본부 임직원수'] = int(html_table[1][1])+int(html_table[1][2])
    except:
        df['가맹본부 임직원수'] = 0
    ## 가맹사업 개시일
    html_table = parser.make2d(tags[5])
    df['가맹사업 개시일']=[html_table[0][1]]
    ## 가맹점 현황
    html_table = parser.make2d(tags[7])
    tmp=pd.DataFrame(html_table[1:], columns=html_table[0])
    df = pd.concat([df,tmp.iloc[:1,1:]],axis=1)
    ## 가맹점 변동현황
    html_table = parser.make2d(tags[8])
    tmp = pd.DataFrame(html_table[2:3], columns=html_table[1])
    df = pd.concat([df,tmp.iloc[:,1:]],axis=1)
    ## 법위반횟수
    html_table = parser.make2d(tags[12])
    total=0
    try:
        for i in html_table[1]:
            total+=int(i)
        df['법위반횟수'] = total
    except:
        df['법위반횟수'] = 0
    ## 가맹점 부담금
    html_table = parser.make2d(tags[13])
    tmp=pd.DataFrame(html_table[1:], columns=html_table[0])
    df = pd.concat([df,tmp],axis=1)
    ## 인테리어 비용
    html_table = parser.make2d(tags[14])
    tmp=pd.DataFrame(html_table[1:], columns=html_table[0])
    df = pd.concat([df,tmp],axis=1)
    
    total_df = pd.concat([total_df,df])

In [65]:
# csv 파일로 변환
total_df.to_csv('data_2020.csv')